# PA005: High Value Customer Identification ( Insiders )

# 0.0 Imports

In [1]:
import re
import sqlite3

import pandas                                                                             as pd
import numpy                                                                              as np
import seaborn                                                                            as sns
import umap.umap_                                                                         as umap


from sqlalchemy                    import create_engine
from datetime                      import datetime

from matplotlib                    import pyplot                                          as plt

from sklearn                       import cluster                                         as c
from sklearn                       import metrics                                         as met
from sklearn                       import decomposition                                   as dd
from sklearn                       import ensemble                                        as en
from sklearn                       import mixture                                         as mx
from plotly                        import express                                         as px
from sklearn                       import preprocessing                                   as pp


import warnings
warnings.filterwarnings("ignore")

##  0.2 Load Dataset


In [2]:
# load data
df_raw = pd.read_csv(r'../data/raw/Ecommerce.csv', encoding='cp1252')

# # drop extra column
df_raw = df_raw.drop(columns=['Unnamed: 8'] , axis=1)

# <font color ='red'> 1.0 Descrição dos dados </font>

In [3]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [4]:
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date','unit_price', 'customer_id', 'country']

df1.columns = cols_new

## 1.2 Data dimensios

In [5]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of columns: {}'.format(df1.shape[1]))

Number of rows: 541909
Number of columns: 8


## 1.3 Data types

In [6]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4 Check NA

In [7]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Replace NA


In [8]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]



In [9]:
# create referance 

df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000 , 19000+len(df_backup), 1)
df_backup.head()

# merge original with reference dataframe
df1 = pd.merge(df1, df_backup , on='invoice_no' , how='left')

#coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# drop extra columns

df1 = df1.drop(columns=['customer_id_x' , 'customer_id_y' ] , axis = 1)

df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,United Kingdom,17850.0
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,United Kingdom,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,United Kingdom,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,United Kingdom,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,United Kingdom,17850.0


## 1.6 Changes dtypes


In [10]:
# invoice date

df1['invoice_date'] = pd.to_datetime(df1['invoice_date'] , format='%d-%b-%y')

# customer_id

df1['customer_id'] = df1['customer_id'].astype('int64')


## 1.7 Descriptive Statistics

In [11]:
num_attributes = df1.select_dtypes(include=['int64' , 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64' , 'float64','datetime64[ns]'])


### 1.7.1 Numerical Attributs

In [12]:
# central tendency - mean, median
mean = pd.DataFrame(num_attributes.apply(np.mean)).T
median = pd.DataFrame(num_attributes.apply(np.median)).T

# dispersion - std , min , max , range , skew , kurtosis
std = pd.DataFrame(num_attributes.apply(np.std)).T
mi = pd.DataFrame(num_attributes.apply(np.min)).T
ma = pd.DataFrame(num_attributes.apply(np.max)).T
ran = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
skew = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
kurtosis = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# concatenate

m = pd.concat([mi , ma ,ran, mean , median  , std,  skew ,  kurtosis]).T.reset_index()
m.columns = ['Attributes' , 'min' , 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtoses']
m

,Attributes,min,max,range,mean,median,std,skew,kurtoses
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


### 1.7.2 Categorical Attributs

#### Invoice No

In [13]:
# identificação:
df_letter_invoices = df1.loc[df1['invoice_no'].apply(lambda x: bool(re.search('[^0-9]+', x))) , :]
df_letter_invoices.head()

print('Total number of invoices: {}'.format(len(df_letter_invoices)))
print('Total number of negative quantity: {}'.format(len(df_letter_invoices['quantity'] < 0)))

Total number of invoices: 9291
Total number of negative quantity: 9291


#### Stock Code

In [14]:
# check stock code only characters
df1.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))),'stock_code'].unique()

# Ação:
## 1. Remove stock_code in ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY','DCGSSGIRL', 'PADS', 'B', 'CRUK']


array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

# <font color ='red'> 2.0 Filtragem de variaveis </font>

In [15]:
df2 = df1.copy()

In [16]:
# ==== Numerical Attributes ====

# unit price > 0.0
df2 = df2.loc[df2['unit_price'] > 0.04 , :]


# ==== Categorical Attributes ====


df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY','DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

# description
df2 = df2.drop('description', axis =1)

# map
df2 = df2[~df2['country'].isin (['European Community' , 'Unspecified'])]

# bad users
df2 = df2[~df2['customer_id'].isin( [16446]) ]


## quantity 
df2_returns = df2.loc[df2['quantity'] < 0, :]
df2_purchase = df2.loc[df2['quantity'] >= 0, :]


# <font color ='red'> 3.0 Feature Engeneering </font>

In [17]:
df3 = df2.copy()

## 3.1 Feature Creation

In [18]:
# data reference

df_ref = df3.drop(['invoice_no','stock_code','quantity', 'invoice_date','unit_price','country'], axis = 1 ).drop_duplicates(ignore_index = True)


### 3.1.1. Gross Revenue

In [19]:
# Gross Revenue ( Faturamento = quantity * price)
df2_purchase.loc[: , 'gross_revenue'] = df2_purchase.loc[: , 'quantity'] * df2_purchase.loc[: , 'unit_price']

df_monetary = df2_purchase.loc[: , ['customer_id' , 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary,on='customer_id',how='left')

df_ref.isna().sum()

customer_id       0
gross_revenue    91
dtype: int64

### 3.1.2. Recency - Day from last purchase

In [20]:
# Recency - Last day purchase
df_recency = df2_purchase.loc[: ,[ 'customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id','recency_days']].copy()
df_ref = pd.merge(df_ref , df_recency , on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
dtype: int64

### 3.1.3. Quantity of purchased

In [21]:
# Numero de produtos
df_freq = (df2_purchase.loc[: ,['customer_id','invoice_no']].drop_duplicates()
                                                            .groupby('customer_id')
                                                            .count()
                                                            .reset_index()
                                                            .rename(columns={'invoice_no' : 'qtde_invoices'}))

df_ref = pd.merge(df_ref , df_freq , on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_invoices    91
dtype: int64

### 3.1.4. Quantity total of items purchased


In [22]:
df_freq = df2_purchase.loc[: ,['customer_id','quantity']].groupby('customer_id').sum().reset_index().rename(columns = {'quantity' : 'qtde_items'})
df_ref = pd.merge(df_ref , df_freq , on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_invoices    91
qtde_items       91
dtype: int64

### 3.1.5. Quantity of products purchased


In [23]:
df_freq = df2_purchase.loc[: ,['customer_id','stock_code']].groupby('customer_id').count().reset_index().rename(columns = {'stock_code' : 'qtde_products'})
df_ref = pd.merge(df_ref , df_freq , on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_invoices    91
qtde_items       91
qtde_products    91
dtype: int64

### 3.1.6. Avg Ticket Value

In [24]:
# Avg ticket
df_avg_ticket = df2_purchase.loc[: ,['customer_id' , 'gross_revenue']].groupby('customer_id').mean().reset_index().rename(columns={'gross_revenue':'avg_ticket'})
#df_avg_ticket['avg_ticket'] = np.round(df_avg_ticket['avg_ticket'] , 2)
df_ref = pd.merge(df_ref, df_avg_ticket,on='customer_id',how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_invoices    91
qtde_items       91
qtde_products    91
avg_ticket       91
dtype: int64

### 3.1.8. Frequency Purchase

In [25]:
df3_purchase = df2_purchase.copy()

In [26]:
df_aux = df3_purchase[['invoice_no','customer_id', 'invoice_date']].drop_duplicates().groupby('customer_id').agg(max_  = ('invoice_date', 'max'), 
                                                                                               min_  = ('invoice_date', 'min'), 
                                                                                               days_ = ('invoice_date', lambda x: (( x.max() - x.min() ).days)+ 1),
                                                                                               buy_  = ('invoice_date', 'count')).reset_index()

# Frequency 
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis = 1)

# Merge
df_ref = pd.merge(df_ref , df_aux[['customer_id', 'frequency']], on='customer_id', how = 'left')

df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_invoices    91
qtde_items       91
qtde_products    91
avg_ticket       91
frequency        91
dtype: int64

### 3.1.9. Numbers of Returns

In [27]:
df_returns = df2_returns[['customer_id' , 'quantity']].groupby('customer_id').sum().reset_index().rename( columns={'quantity' : 'qtde_returns'})
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1
df_ref = pd.merge(df_ref , df_returns, on ='customer_id' ,  how='left')

df_ref.loc[df_ref['qtde_returns'].isna() , 'qtde_returns'] = 0
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_invoices    91
qtde_items       91
qtde_products    91
avg_ticket       91
frequency        91
qtde_returns      0
dtype: int64

# <font color ='red'> 4.0 EDA (Exploratory Data Analysis) </font>

In [28]:
df4 = df_ref.dropna().copy()

## 4.3 Estudo do espaço

In [29]:
# Selected Dataset
cols_selected = ['customer_id', 'gross_revenue','recency_days', 'qtde_products', 'frequency', 'qtde_returns' ]
df43 = df4[cols_selected].drop(columns = 'customer_id')


In [30]:
mm = pp.MinMaxScaler()

df43['gross_revenue']          = mm.fit_transform(df43[['gross_revenue']])
df43['recency_days']           = mm.fit_transform(df43[['recency_days']])
df43['qtde_products']          = mm.fit_transform(df43[['qtde_products']])
df43['frequency']              = mm.fit_transform(df43[['frequency']])
df43['qtde_returns']           = mm.fit_transform(df43[['qtde_returns']])


X = df43.copy()


In [31]:
X.shape

(5695, 5)

### 4.3.4 Tree-Based Embedding

não precisar fazer o minmaxscaler() para trabalhar com uma arvore

In [33]:
# training dataset 
X = df43.drop(columns=[ 'gross_revenue'], axis = 1)
Y = df43['gross_revenue']

# model definition
rf_model = en.RandomForestRegressor(n_estimators = 100 , random_state = 42 )

# model training
rf_model.fit(X,Y)

# dataframe Leaf
df_leaf = pd.DataFrame(rf_model.apply(X))

In [34]:
# reduzer dimensionality
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(df_leaf)


# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:,0]
df_tree['embedding_y'] = embedding[:,1]



In [35]:
df5 = df_tree.copy()

In [36]:
df6 = df_tree.copy()

# <font color ='red'> 5.0 Hyperparameter Fine-Tunning </font>

In [37]:
x = df_tree.copy()

# <font color ='red'> 6.0 Model Training </font>

In [38]:
df8 = x.copy()

## 6.1 GMM

In [39]:
k = 9

# model definition
k = 11
kmeans = c.KMeans(init='random', n_clusters=k, n_init=10, max_iter=300, random_state=42)

# model training
kmeans.fit(df8)

# clustering
labels = kmeans.labels_


## 6.2 Cluster Validation

In [40]:
print('SS value: {}'.format(met.silhouette_score(df8, labels, metric='euclidean')))

SS value: 0.4246853291988373


# <font color ='red'> 7.0 Cluster Analysis </font>

In [41]:
df92 = df4[cols_selected].copy()
df92['cluster'] = labels

# change dtypes
df92['recency_days'] = df92['recency_days'].astype('int64')
df92['qtde_products'] = df92['qtde_products'].astype('int64')
df92['qtde_returns'] = df92['qtde_returns'].astype('int64')

df92['last_training_timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [42]:
df92.head()

,customer_id,gross_revenue,recency_days,qtde_products,frequency,qtde_returns,cluster,last_training_timestamp
0,17850,5391.21,372,297,17.000000,40,1,2021-12-04 22:08:50
1,13047,3232.59,56,171,0.028302,35,1,2021-12-04 22:08:50
2,12583,6705.38,2,232,0.040323,50,1,2021-12-04 22:08:50
3,13748,948.25,95,28,0.017921,0,2,2021-12-04 22:08:50
4,15100,876.00,333,3,0.073171,22,8,2021-12-04 22:08:50


## 7.2 Cluster Profile


In [43]:
# Number of customer
df_cluster = df92[['customer_id','cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customers'] = 100*(df_cluster['customer_id'] / df_cluster['customer_id'].sum())

# Avg Gross revenue
df_avg_gross_revenue = df92[['gross_revenue','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster , df_avg_gross_revenue , how='inner' , on='cluster')

# Avg recency days
df_avg_recency_days = df92[['recency_days','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster , df_avg_recency_days , how='inner' , on='cluster')

# Avg invoice_no
df_qtde_products = df92[['qtde_products','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster , df_qtde_products , how='inner' , on='cluster')

# Frequency
df_frequency = df92[['frequency','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster , df_frequency , how='inner' , on='cluster')

# Returns
df_qtde_returns = df92[['qtde_returns','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster , df_qtde_returns , how='inner' , on='cluster')


df_cluster

,cluster,customer_id,perc_customers,gross_revenue,recency_days,qtde_products,frequency,qtde_returns
0,0,226,3.968393,2314.579071,81.486726,46.376106,0.831506,6.880531
1,1,1038,18.226514,5488.149355,64.015414,294.214836,0.298030,149.525048
2,2,194,3.406497,633.885103,133.185567,20.221649,0.301263,2.386598
3,3,1366,23.985953,511.448353,157.079063,34.316252,0.948472,5.008785
4,4,872,15.311677,1095.678062,124.735092,56.933486,0.533552,4.645642
5,5,678,11.905180,1924.082566,88.382006,121.290560,0.381486,3.883481
6,6,361,6.338894,1252.126482,60.268698,54.382271,0.027892,8.975069
7,7,508,8.920105,445.873917,127.013780,11.576772,0.445041,1.200787
8,8,452,7.936787,275.281925,189.517699,7.247788,0.675771,7.681416


In [44]:
# 4 cluster Insiders
# 8 cluster more products
# 5 cluster spend money
# 0 cluster even more products
# 6 cluster less days
# 3 cluster less 1k
# 1 cluster Stop Returners
# 2 cluster more buy
# 7 cluster even more bu6

### Cluster 01: ( Candidato a Insiders)

    - Número de customers: 468 (16% do customers)
    - Faturamento médio: 8836
    - Recência média: 21 dias 
    - Média de produtos comprados: 424 produtos
    - Frequência de produtos comprados: 0.09 produtos/dia
    - Receita em média: $ 8835.90,00 dólares
    
### Cluster 02: 

    - Número de customers: 31 (0.71% do customers)
    - Recência em média: 14 dias 
    - Compras em média: 53 compras
    - Receita em média: $ 40.543,52 dólares

### Cluster 03: 

    - Número de customers: 4.335 (99.15% do customers)
    - Recência em média: 92 dias 
    - Compras em média: 5 compras
    - Receita em média: $ 1.372,57 dólares

# <font color ='red'> 8.0 Deploy to production </font>

In [45]:
df92.dtypes

customer_id                  int64
gross_revenue              float64
recency_days                 int64
qtde_products                int64
frequency                  float64
qtde_returns                 int64
cluster                      int64
last_training_timestamp     object
dtype: object

## 8.1. Insert into SQLITE

In [46]:
# create Table
# query_create_table_insiders = """
#     CREATE TABLE Insiders (
#         customer_id        INTEGER,
#         gross_revenue      REAL,
#         recency_days       INTEGER,
#         qtde_products      INTEGER,
#         frequency          REAL,
#         qtde_returns       INTEGER,
#         cluster            INTEGER
#      )

# """

# conn = sqlite3.connect('insiders_db.sqlite')
# conn.execute(query_create_table_insiders)
# conn.commit()
# conn.close()

# insert data
conn = create_engine('sqlite:///insiders_db.sqlite')
# df92.to_sql('insiders' , con = conn, if_exists='append', index=False)

# select data 

In [47]:
# consulting database
query = """
    SELECT * FROM Insiders
"""


df = pd.read_sql_query(query , conn)

In [48]:
df.head()

,customer_id,gross_revenue,recency_days,qtde_products,frequency,qtde_returns,cluster
0,17850,5391.21,372,297,17.000000,40,4
1,13047,3232.59,56,171,0.028302,35,5
2,12583,6705.38,2,232,0.040323,50,4
3,13748,948.25,95,28,0.017921,0,1
4,15100,876.00,333,3,0.073171,22,7


In [49]:
df.shape

(2968, 7)